In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import plotly.express as px

# Air pollution and asthma hospitalizations in children. What does data visualization tell us?

It is almost general knoweldge that air pollution might be one of the main causes of respiratory diseases. An among all the broad types of respiratory diseases that exists, specifically asthma. Asthma is a chronic respiratory disease characterized by variable airflow obstruction, bronchial hyperresponsiveness, and airway inflammation. [[1]](https://pubmed.ncbi.nlm.nih.gov/32867076/) Researchers have long linked asthma with exposure to air pollution, which can make asthma symptoms worse and trigger asthma attacks. Moreover, it is estimated six million children in the United States with asthma are especially vulnerable to air pollution. [[2]](https://www.epa.gov/sciencematters/links-between-air-pollution-and-childhood-asthma#:~:text=Researchers%20have%20long%20linked%20asthma,worse%20and%20trigger%20asthma%20attacks). And not only vulnerable once they have the disease. It has been also proved that exposure to main air pollutants such as NO2, CO, and PM2.5 is linked to regional DNA methylation differences in asthma. [[3]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5756438/pdf/13148_2017_Article_433.pdf)

The aim of this study is to dig deeper into this relation between air pollution and asthma. To do so, NYC air pollution data will be used and combined with asthma hospitalizations data for children. Is there any clear correlation between air pollution and the number of asthma hospitalizations? If so, which are the air pollutants that are more correlated? How do these values vary between the different neighbourhoods? Where is more urgent to reduce the emissions of hazardous air pollutants? Is it possible to predict which is gonna be the number of asthma hospitalizations in order to prepare these hospitals that are gonna suffer more for such big workloads? 

In [3]:
data=pd.read_csv('machine_learning_data_Final.csv')
data=data.iloc[:,1:]
data.head()

,year,district,NO2,PM2_5,O3,SO2,asthma_hosp,Boroughs
0,2009,101,23.20,11.03,23.67,6.62,61,Bronx
1,2009,102,22.39,10.68,26.82,5.38,282,Bronx
2,2009,103,24.82,11.10,24.47,9.48,693,Bronx
3,2009,104,22.83,10.59,26.72,5.15,588,Bronx
4,2009,105,28.07,11.76,23.08,9.36,631,Bronx


### Line Starting plot

In [22]:
def rep_array(array,nrep=5):
    repetitions=[]; torep=list(array)
    for i in range(nrep):
        repetitions=repetitions+torep
    return repetitions

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


"""def NormalizeData(array):
    #norm = np.linalg.norm(array)
    norm = np.mean(array)
    return array/norm"""

"""def NormalizeData(array):
    return array"""

# let's extract the mean astham hosp. values per year
asthma_per_year=NormalizeData([data[data.year==el].asthma_hosp.mean() for el in np.unique(data.year)])

# Mean pollutants per year
PM2_5_per_year=NormalizeData([data[data.year==el].PM2_5.mean() for el in np.unique(data.year)])
NO2_per_year=NormalizeData([data[data.year==el].NO2.mean() for el in np.unique(data.year)])
O3_per_year=NormalizeData([data[data.year==el].O3.mean() for el in np.unique(data.year)])
SO2_per_year=NormalizeData([data[data.year==el].SO2.mean() for el in np.unique(data.year)])

years=np.unique(data.year)

labels=[np.repeat('PM2_5',8),np.repeat('NO2',8),np.repeat('O3',8),np.repeat('SO2',8),
        np.repeat('Asthma Hospitalizations',8)]

labels=np.hstack(labels)
y=rep_array(years)
values=np.hstack([PM2_5_per_year,NO2_per_year,O3_per_year,SO2_per_year,asthma_per_year])
values

line_plot_df=pd.DataFrame({'year':y,'value':values,'label':labels})

In [23]:
line_plot_df_pollutants=line_plot_df.iloc[:-8,:]

In [24]:
# general line plot                         
fig = px.line(line_plot_df, x='year', y='value', color='label', 
              title='Evolution of Air Pollution and Children Asthma Hospitalizations in NYC')
fig.show()

In [25]:
# general line plot                         
fig = px.line(line_plot_df, x='year', y='value', color='label', 
              title='Evolution of Air Pollution and Children Asthma Hospitalizations in NYC')
fig.show()

In [26]:
years=[9,10,11,12,13,14,15,16]

In [27]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=5,column_widths=[0.4, 0.15,0.15,0.15,0.15],
    subplot_titles=("Asthma hosp","PM2.5", "NO2", "O3", "SO2"))

fig.add_trace(go.Scatter(x=years, y=asthma_per_year,name='Asthma hosp'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=years, y=PM2_5_per_year,name='PM2.5'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=years, y=NO2_per_year,name='NO2'),
              row=1, col=3)

fig.add_trace(go.Scatter(x=years, y=O3_per_year,name='O3'),
              row=1, col=4)

fig.add_trace(go.Scatter(x=years, y=SO2_per_year,name='SO2'),
              row=1, col=5)

# Update xaxis properties
fig.update_xaxes(title_text="year", row=1, col=1)
fig.update_xaxes(title_text="year", row=1, col=2)
fig.update_xaxes(title_text="year", row=1, col=3)
fig.update_xaxes(title_text="year", row=1, col=4)
fig.update_xaxes(title_text="year", row=1, col=5)


fig.update_layout(height=350, width=1000,
                  title_text='Evolution of Air Pollution and Children Asthma Hospitalizations in NYC')

fig.update_layout(showlegend=False)
fig.show()

The first step has been to combine both datasets per year and district. Thus the average NO2, PM2.5, O3 and SO2 values have been computed for each category and saved together with the number of hospitalizations. First thing to study then is which is the correlation between the different air pollutants, and if one of those show any correlation with the numer of asthma hospitalizations.

In [30]:
data=pd.read_csv('Machine_Model_Data.csv')
data=data.iloc[:,1:]
data

,year,district,age_group,NO2,PM2_5,O3,SO2,asthma_hosp
0,2009,101,0-4y,23.20,11.03,23.67,6.62,30
1,2009,101,5-17y,23.20,11.03,23.67,6.62,31
2,2009,102,0-4y,22.39,10.68,26.82,5.38,155
3,2009,102,5-17y,22.39,10.68,26.82,5.38,127
4,2009,103,0-4y,24.82,11.10,24.47,9.48,349
...,...,...,...,...,...,...,...,...
667,2016,502,5-17y,14.93,6.80,34.18,0.13,31
668,2016,503,0-4y,14.11,7.10,34.68,0.12,12
669,2016,503,5-17y,14.11,7.10,34.68,0.12,0
670,2016,504,0-4y,11.52,6.59,36.17,0.11,24


Not really interested in separating between age_groups, let's merge both rows per year and district.

In [29]:
df=data.copy()
for i in range(df.shape[0]):
    if i%2==0:
        asthma=0
        asthma=df.asthma_hosp[i]
        df = df.drop(labels=i, axis=0)
    else:
        asthma+=df.asthma_hosp[i]
        df.loc[i,'asthma_hosp']=asthma
df=df.reset_index()
df=df.iloc[:,1:]
df=df.drop(labels='age_group',axis=1)
df

,year,district,NO2,PM2_5,O3,SO2,asthma_hosp
0,2009,101,23.20,11.03,23.67,6.62,61
1,2009,102,22.39,10.68,26.82,5.38,282
2,2009,103,24.82,11.10,24.47,9.48,693
3,2009,104,22.83,10.59,26.72,5.15,588
4,2009,105,28.07,11.76,23.08,9.36,631
...,...,...,...,...,...,...,...
331,2016,410,11.72,5.98,38.18,0.20,73
332,2016,501,17.47,7.30,33.42,0.12,83
333,2016,502,14.93,6.80,34.18,0.13,72
334,2016,503,14.11,7.10,34.68,0.12,12


In [20]:
# df.to_csv('machine_learning_data_Final.csv')

In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

air_pollutants=['NO2','PM2_5','O3','SO2']
plot_idxs=[[1,1],[1,2],[2,1],[2,2]]

fig = make_subplots(rows=2, cols=2, start_cell="top-left",subplot_titles=('NO2','PM2_5','O3','SO2'))

for idx,el in enumerate(air_pollutants):
    fig.add_trace(go.Scatter(x=df[el], y=df["asthma_hosp"],
                            mode="markers",name=el)
                  ,row=plot_idxs[idx][0], col=plot_idxs[idx][1])
    fig.update_xaxes(title_text=el,row=plot_idxs[idx][0], col=plot_idxs[idx][1])
    fig.update_yaxes(title_text="Asthma hosp")

fig.show()

In [353]:
air_pollutants=['NO2','PM2_5','O3','SO2']
fig = px.scatter(df, x='PM2_5', y="asthma_hosp", trendline="ols",color="year")
fig.show()

### More PM2.5 values do not strictly mean more asthma hospitalizations

Contradicting the results found in Corinne A Keet et al. study [[3]](https://pubmed.ncbi.nlm.nih.gov/29243937/) it cannot be affirmed that long-term PM2-5 exposure is associated with asthma among children in medicaid.

It can be observed how the levels of PM2.5 have significantly decrease by year. But even this reduction, the number of hospitalizations still remains quite high during the years. 

However, it is true that the results are highly affected by the huge amount of 0 or low values in terms of asthma hospitalizations. This might be due to the fact that these hospitals might not be able to handle asthma patients, and they are redirected to another hospital. Let's dig more into this phenomenon to find which may be the cause of this effect, by first filtering for just those hospitals that recieve more tha 50 average asthma visits per year.

In [121]:
df_non_0=df[df.asthma_hosp>75]
fig = px.scatter(df_non_0, x='PM2_5', y="asthma_hosp", trendline="ols",color="year")
fig.show()

Let's add the broroughs of each district as an extra column in the dataframe in order to facet the plots.

In [126]:
nyc_ares=['Bronx','Brooklyn','Manhattan','Queens','Staten Island']
df['Boroughs']=[nyc_ares[int(str(el)[0])-1] for el in df.district]
df.head()

,year,district,NO2,PM2_5,O3,SO2,asthma_hosp,Boroughs
0,2009,101,23.20,11.03,23.67,6.62,61,Bronx
1,2009,102,22.39,10.68,26.82,5.38,282,Bronx
2,2009,103,24.82,11.10,24.47,9.48,693,Bronx
3,2009,104,22.83,10.59,26.72,5.15,588,Bronx
4,2009,105,28.07,11.76,23.08,9.36,631,Bronx


In [170]:
fig = px.scatter(df, x="year", y="asthma_hosp",trendline="ols")
fig.show()

In [171]:
fig = px.scatter(df, x="PM2_5", y="asthma_hosp", color="year",trendline="ols", facet_col="Boroughs")
fig.show()

### When the trendline highly varies depending on the district we are located
Something really interesting can be observed here, and is the fact that really different regression trendlines can be observed depending on the neighbourhood. While in the Bronx the number of asthma hospitalizations strongly increases as the levels of PM2.5 increase, in Manhattan happens exactly the opposite.

Bronx, Brookykn and Queens district seem to show the expected behaviour. As levels of PM2.5 increase, air becomes more polluted, and so the number of asthma hospitalizations increases. However, why does the number of hospitalizations decrease in Manhattan as the levels of air pollution increase?

Different hypothesis appear here. Is it because in the more polluted areas of Manhattan, there is less space for hospitals in which you can treat asthma cases? Or is it because healtcare and quality of life is basically better in Mantattan?

Is it hard to believe that the first hypothesis might be the cause, specially becuase if we look at the more polluted ares of Manhattan we can find hospitals such as the Metropolitan [[4]](https://www.nychealthandhospitals.org/metropolitan/about-us/), which have a Children’s Asthma Program or Bellevue [[5]](https://www.nychealthandhospitals.org/bellevue/health-care-services/childrens-health/) which provides multidisciplinary care for children and adolescents with asthma.

Thus, it seems that this tendency is due to the fact

If we go to check the Median Household Income on 2017 in NYC, we observe that the Bronx has the lowest value with a Median Household Income of 37.397 \\$ , while Manhattan has a value up to 85.071 \\$.

In [154]:
df.head()

,year,district,NO2,PM2_5,O3,SO2,asthma_hosp,Boroughs
0,2009,101,23.20,11.03,23.67,6.62,61,Bronx
1,2009,102,22.39,10.68,26.82,5.38,282,Bronx
2,2009,103,24.82,11.10,24.47,9.48,693,Bronx
3,2009,104,22.83,10.59,26.72,5.15,588,Bronx
4,2009,105,28.07,11.76,23.08,9.36,631,Bronx


In [163]:
df_bronx=df[df.Boroughs=="Bronx"]
fig = px.scatter(df_bronx, x="PM2_5", y="asthma_hosp",trendline="ols", facet_col="Boroughs",color="year")
fig.show()

In [165]:
df_manhattan=df[df.Boroughs=="Manhattan"]
fig = px.scatter(df_manhattan, x="PM2_5", y="asthma_hosp",trendline="ols",color="year")
fig.show()

In [152]:
df_2016=df[df.year>2015]
fig = px.scatter(df_2016, x="NO2", y="asthma_hosp",trendline="ols", facet_col="Boroughs")
fig.show()

In [140]:
df

,year,district,NO2,PM2_5,O3,SO2,asthma_hosp,Boroughs
0,2009,101,23.20,11.03,23.67,6.62,61,Bronx
1,2009,102,22.39,10.68,26.82,5.38,282,Bronx
2,2009,103,24.82,11.10,24.47,9.48,693,Bronx
3,2009,104,22.83,10.59,26.72,5.15,588,Bronx
4,2009,105,28.07,11.76,23.08,9.36,631,Bronx
...,...,...,...,...,...,...,...,...
331,2016,410,11.72,5.98,38.18,0.20,73,Queens
332,2016,501,17.47,7.30,33.42,0.12,83,Staten Island
333,2016,502,14.93,6.80,34.18,0.13,72,Staten Island
334,2016,503,14.11,7.10,34.68,0.12,12,Staten Island


In [135]:
df_non_0=df[df.asthma_hosp>50]
fig = px.scatter(df_non_0, x="PM2_5", y="asthma_hosp", color="year",trendline="ols", facet_col="Boroughs")
fig.show()

In [136]:
fig = px.scatter(df, x="O3", y="asthma_hosp", color="year",trendline="ols", facet_col="Boroughs")
fig.show()

In [137]:
fig = px.scatter(df, x="SO2", y="asthma_hosp", color="year",trendline="ols", facet_col="Boroughs")
fig.show()

In [139]:
fig = px.scatter(df, x="NO2", y="asthma_hosp", color="year",trendline="ols", facet_col="Boroughs")
fig.show()